<a href="https://colab.research.google.com/github/merucode/RL/blob/92-Project-Trader-Custom_environment/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 1. Install Dependency

In [1]:
!git clone -b 92-Project-Trader-Custom_environment https://github.com/merucode/RL.git
!cd RL && mv * ../

Cloning into 'RL'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 132 (delta 20), reused 0 (delta 0), pack-reused 80
Receiving objects: 100% (132/132), 8.64 MiB | 5.28 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [7]:
!pip install -e trader_world

Obtaining file:///content/trader_world
  Preparing metadata (setup.py) ... done
  Attempting uninstall: trader-world
    Found existing installation: trader-world 0.0.1
    Uninstalling trader-world-0.0.1:
      Successfully uninstalled trader-world-0.0.1
  Running setup.py develop for trader-world


### NOTE: After inistall gym-examples, Restart runtime

# STEP 2. Ready Data

### Data From csv

In [1]:
import pandas as pd

df1 = pd.read_csv('KRW-BTC_5m_171001_200903_upbit.csv')
df2 = pd.read_csv('KRW-BTC_5m_200904_230810_upbit.csv')

df = pd.concat([df1, df2], ignore_index=True)
df = df[df.columns[1:]]
print(df)

              open        high         low       close    volume
0        4797000.0   4800000.0   4793000.0   4800000.0  0.013444
1        4798000.0   4799000.0   4792000.0   4793000.0  0.013597
2        4795000.0   4802000.0   4795000.0   4795000.0  0.016304
3        4801000.0   4802000.0   4795000.0   4801000.0  0.017136
4        4797000.0   4799000.0   4791000.0   4799000.0  0.007615
...            ...         ...         ...         ...       ...
492907  39205000.0  39208000.0  39179000.0  39206000.0  2.508436
492908  39206000.0  39212000.0  39186000.0  39212000.0  1.350777
492909  39190000.0  39231000.0  39190000.0  39212000.0  2.929709
492910  39212000.0  39214000.0  39188000.0  39191000.0  3.407502
492911  39191000.0  39210000.0  39188000.0  39188000.0  3.884740

[492912 rows x 5 columns]


In [2]:
# Preporcessing
df_ohlc = df[df.columns[:4]]/10000
df = pd.concat([df_ohlc, df[df.columns[-1:]]], axis=1)
df = df[:1000]  # For test
print(df)

      open   high    low  close    volume
0    479.7  480.0  479.3  480.0  0.013444
1    479.8  479.9  479.2  479.3  0.013597
2    479.5  480.2  479.5  479.5  0.016304
3    480.1  480.2  479.5  480.1  0.017136
4    479.7  479.9  479.1  479.9  0.007615
..     ...    ...    ...    ...       ...
995  488.8  489.0  485.4  488.6  0.015235
996  488.6  488.8  485.1  488.4  0.016009
997  489.1  489.1  485.2  488.3  0.013057
998  485.6  489.1  485.5  488.4  0.010872
999  488.0  488.8  488.0  488.3  0.009211

[1000 rows x 5 columns]


### [Not use]Reference get data code

In [ ]:
#!pip install pyupbit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
### Ready Data
import time
from datetime import datetime, timedelta


import pyupbit

ticker = 'KRW-BTC'
interval = 'minute5'
start_date = datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')

df_to = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])

for _ in range(30):
    to = datetime.strftime(start_date, "%Y-%m-%d %H:%M:%S")
    count = 180
    df = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)
    df = df[df.columns[:-1]]
    start_date += timedelta(hours=15) # hours=3(1m), hours=15(5m)

    df_to = pd.concat([df_to,df])
    time.sleep(0.5)

# print(df_to)
df_to = df_to.reset_index(drop=True)
df = df_to[df_to.columns[:4]]/10000
df = pd.concat([df, df_to[df_to.columns[-1:]]], axis=1)
print(df)

        open    high     low   close     volume
0     3199.2  3199.7  3195.7  3195.9  17.041450
1     3195.9  3197.6  3193.3  3194.3  18.890478
2     3193.3  3194.3  3179.8  3180.8  50.900129
3     3180.8  3188.4  3175.1  3187.3  61.697074
4     3188.0  3197.5  3186.1  3195.2  18.563247
...      ...     ...     ...     ...        ...
5395  4017.9  4022.1  4015.5  4022.0  15.316342
5396  4022.0  4022.4  4008.0  4008.0  36.836909
5397  4007.8  4015.2  4002.2  4015.2  27.802065
5398  4009.7  4018.5  4009.7  4014.1  38.511329
5399  4014.1  4019.5  4011.9  4019.5  27.235054

[5400 rows x 5 columns]


# STEP 3. Setting ENV

In [9]:
import time
import matplotlib.pyplot as plt
import gym

#env = gym.make("trader_world:TraderWorld-v0", df=df, df_render=df, obs_len=180, actions=180, render_mode="rgb_array")
env = gym.make("trader_world:TraderWorld-v0", df=df, df_render=df, obs_len=180, actions=12)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (180, 5)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# single env
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'observation shape : {env.observation_space.shape[0]}, {env.observation_space.shape[1]}')
print(f'action shape      : {env.action_space.n}')

In [10]:
reset_obs = env.reset()
print(reset_obs)
#env.action_space, env.observation_space
#action = env.action_space.sample()
#action
#obs, reward, done, info = env.step(5)
#print(reward, done, info)
#img = env.render()
#plt.figure()
#plt.imshow(img)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


In [16]:
# single env
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'observation shape : {env.observation_space.shape[0]}, {env.observation_space.shape[1]}')
print(f'action shape      : {env.action_space.n}')

observation space : Box(0, 9223372036854775805, (180, 5), int64)
action space      : Discrete(12)
observation shape : 180, 5
action shape      : 12


### Multi Env

In [3]:
import gym
from gym.wrappers import RecordVideo, RecordEpisodeStatistics, \
                         NormalizeObservation, NormalizeReward

def create_env(env_name, num_envs, **kwargs):
    env = gym.vector.make(env_name, num_envs=num_envs, **kwargs)
    #env = RecordEpisodeStatistics(env)
    #env = NormalizeObservation(env)  # because of volume, it is not working well
    #env = NormalizeReward(env)
    return env

In [14]:
env = create_env("trader_world:TraderWorld-v0", num_envs=2, df=df, df_render=df, obs_len=180, actions=12)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `reset()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:226: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


In [15]:
# multi env
print(f'env number        : {env.num_envs}')
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'observation shape : {env.observation_space.shape}')
print(f'obesrvaiotn each shpae : {env.observation_space.shape[0]}, {env.observation_space.shape[1]}, {env.observation_space.shape[2]}')
print(f'single obesrvaiotn shpae : {env.single_observation_space.shape[0]}, {env.single_observation_space.shape[1]}')
print(f'action shape      : {env.action_space.shape}')
print(f'action each shape : {env.action_space.shape[0]}')
print(f'single action space : {env.single_action_space.n}')

env number        : 2
observation space : Box(0.0, inf, (2, 180, 5), float64)
action space      : MultiDiscrete([12 12])
observation shape : (2, 180, 5)
obesrvaiotn each shpae : 2, 180, 5
single obesrvaiotn shpae : 180, 5
action shape      : (2,)
action each shape : 2
single action space : 12


In [16]:
reset_obs = env.reset()
print(reset_obs)

[[[4.7970000e+02 4.8000000e+02 4.7930000e+02 4.8000000e+02 1.3444000e-02]
  [4.7980000e+02 4.7990000e+02 4.7920000e+02 4.7930000e+02 1.3597000e-02]
  [4.7950000e+02 4.8020000e+02 4.7950000e+02 4.7950000e+02 1.6304000e-02]
  ...
  [4.8710000e+02 4.8820000e+02 4.8700000e+02 4.8700000e+02 1.2523321e-01]
  [4.8750000e+02 4.8750000e+02 4.8700000e+02 4.8700000e+02 9.8770000e-03]
  [4.8750000e+02 4.8800000e+02 4.8690000e+02 4.8690000e+02 3.9949515e-01]]

 [[4.7970000e+02 4.8000000e+02 4.7930000e+02 4.8000000e+02 1.3444000e-02]
  [4.7980000e+02 4.7990000e+02 4.7920000e+02 4.7930000e+02 1.3597000e-02]
  [4.7950000e+02 4.8020000e+02 4.7950000e+02 4.7950000e+02 1.6304000e-02]
  ...
  [4.8710000e+02 4.8820000e+02 4.8700000e+02 4.8700000e+02 1.2523321e-01]
  [4.8750000e+02 4.8750000e+02 4.8700000e+02 4.8700000e+02 9.8770000e-03]
  [4.8750000e+02 4.8800000e+02 4.8690000e+02 4.8690000e+02 3.9949515e-01]]]


# STEP 4. Reinforce

In [17]:
!apt-get install -y xvfb

!pip install \
  pygame \
  gym==0.25.2 \
  pytorch-lightning==1.6 \
  pyvirtualdisplay

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 16 not upgraded.
Need to get 7,812 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [4]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import copy
import torch
import random
import gym
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/__init__.py:38: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pytorch_lightning')

In [6]:
def test_env(env_name, policy, obs_rms):
    env = gym.make(env_name)
    env = RecordVideo(env, 'videos', episode_trigger=lambda e: True)
    env = NormalizeObservation(env)
    env.obs_rms = obs_rms

    for episode in range(10):
        done = False
        obs = env.reset()
        while not done:
            action = policy(obs).multinomial(1).cpu().item()
            obs, _, done, _ = env.step(action)
    del env


def display_video(episode=0):
    video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"")

In [7]:
class GradientPolicy(nn.Module):

    def __init__(self, in_features, n_actions, hidden_size=128):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, n_actions)

    def forward(self, x):
        x = torch.tensor(x).float().to(device)
        x = torch.tensor(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=-1)
        return x

In [8]:
policy = GradientPolicy(180*5, 12)

In [5]:
envs = create_env("trader_world:TraderWorld-v0", num_envs=2, df=df, df_render=df, obs_len=180, trade_actions=12)
#env = gym.make("trader_world:TraderWorld-v0", df=df, df_render=df, obs_len=180, trade_actions=12)
#actions = 1
envs.reset()
actions = np.array([0, 0])
next_obs, rewards, dones, infos = envs.step(actions)
print(f'next_obs : {next_obs}')
print(f'rewards  : {rewards}')
print(f'dones    : {dones}')
print(f'infos    : {infos}')

# TypeError: object of type 'NoneType' has no len()
# Solution : step return의 observation -> np.array(observation, dtype=np.float32),
# actions -> trade_actions

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (180, 5)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/vector/vector_env.py:56: DeprecationWarning: WARN: Initializing vector env in old step API which returns one bool array instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:137: UserWarning: WARN:

DEBUG POINT step_returns: None None


TypeError: ignored

In [14]:
#envs = gym.vector.make("CartPole-v1", num_envs=3)
envs = gym.vector.make("trader_world:TraderWorld-v0", num_envs=3, df=df, df_render=df, obs_len=180, actions=12)
envs.reset()
actions = np.array([1, 0, 1])
observations, rewards, dones, infos = envs.step(actions)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (180, 5)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/vector/vector_env.py:56: DeprecationWarning: WARN: Initializing vector env in old step API which returns one bool array instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.


TypeError: ignored

In [24]:
lst = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]
print(lst)

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
np.array(lst, dtype=float)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[ 1.,  2.,  3.,  4.,  5.],
       [ 6.,  7.,  8.,  9., 10.]])